In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
%cd '/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset'

/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset


In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
import shap

df_INS_W1 = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_1_sleep_step_screen_allday_100_participants.csv')
df_INS_W2 = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_2_sleep_step_screen_allday_100_participants.csv')
df_INS_W3 = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_3_sleep_step_screen_allday_100_participants.csv')
df_INS_W4 = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/INS-W_4_sleep_step_screen_allday_100_participants.csv')

In [ ]:
print(df_INS_W1.shape)
print(df_INS_W2.shape)
print(df_INS_W3.shape)
print(df_INS_W4.shape)

(7313, 199)
(7580, 199)
(7894, 199)
(8197, 199)


In [ ]:
df_INS_W2.head()

,pid,date,f_screen:phone_screen_rapids_avgdurationunlock:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_exercise_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_greens_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_home_norm:allday,f_screen:phone_screen_rapids_avgdurationunlock_locmap_living:allday,...,f_steps:fitbit_steps_summary_rapids_avgsumsteps_norm:allday,f_steps:fitbit_steps_summary_rapids_maxsumsteps:allday,f_steps:fitbit_steps_summary_rapids_maxsumsteps_norm:allday,f_steps:fitbit_steps_summary_rapids_mediansumsteps:allday,f_steps:fitbit_steps_summary_rapids_mediansumsteps_norm:allday,f_steps:fitbit_steps_summary_rapids_minsumsteps:allday,f_steps:fitbit_steps_summary_rapids_minsumsteps_norm:allday,f_steps:fitbit_steps_summary_rapids_stdsumsteps:allday,f_steps:fitbit_steps_summary_rapids_stdsumsteps_dis:allday,f_steps:fitbit_steps_summary_rapids_stdsumsteps_norm:allday
0,INS-W_303#INS-W_2,2019-03-21,35.737283,0.999983,0.0,0.999983,0.000000,2.997503,0.00000,2.578566,...,0.0,10103.5,0.0,10103.5,0.0,10103.5,0.0,NaN,NaN,NaN
1,INS-W_303#INS-W_2,2019-03-22,2.902666,0.999983,0.0,0.999983,0.000000,2.997503,0.00000,2.578566,...,0.0,10103.5,0.0,10103.5,0.0,10103.5,0.0,NaN,NaN,NaN
2,INS-W_303#INS-W_2,2019-03-23,1.152253,0.999983,0.0,0.999983,0.000000,2.997503,0.00000,2.578566,...,0.0,10103.5,0.0,10103.5,0.0,10103.5,0.0,NaN,NaN,NaN
3,INS-W_303#INS-W_2,2019-03-24,7.404358,0.999983,0.0,0.999983,0.000000,2.997503,0.00000,2.578566,...,0.0,10103.5,0.0,10103.5,0.0,10103.5,0.0,NaN,NaN,NaN
4,INS-W_303#INS-W_2,2019-03-25,1.262568,0.999983,0.0,1.103491,0.330679,1.516245,-0.06703,2.578566,...,0.0,10103.5,0.0,10103.5,0.0,10103.5,0.0,NaN,NaN,NaN


In [ ]:
import pandas as pd

# Copy of the sleep data
sleep_data = df_INS_W4.copy()

# Ensure all columns are numeric for correlation calculation
numeric_data = sleep_data.select_dtypes(include=['number'])

# Calculate correlations with the target variable
target_column = 'f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday'
if target_column in numeric_data.columns:
    correlation_matrix = numeric_data.corr()
    target_correlation = correlation_matrix[target_column]

    # Sort correlations in descending order (excluding the target variable itself)
    top_features_correlation = target_correlation.drop(
        target_column
    ).abs().sort_values(ascending=False).head(7)

    print("Top 7 features based on correlation:")
    print(top_features_correlation)
else:
    print(f"Error: Target column '{target_column}' is not numeric or missing.")


Top 7 features based on correlation:
f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain:allday                     0.918072
f_slp:fitbit_sleep_summary_rapids_avgdurationawakemain:allday                     0.917659
f_slp:fitbit_sleep_summary_rapids_avgefficiencymain_norm:allday                   0.552259
f_slp:fitbit_sleep_intraday_rapids_sumdurationawakeunifiedmain:allday             0.546652
f_slp:fitbit_sleep_intraday_rapids_ratiodurationasleepunifiedwithinmain:allday    0.524272
f_slp:fitbit_sleep_intraday_rapids_ratiodurationawakeunifiedwithinmain:allday     0.524272
f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain_norm:allday                0.522296
Name: f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday, dtype: float64


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np

# Define relevant features (activity, screen time, etc.)
sleep_data = df_INS_W4.copy()
selected_features = [
    'f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain:allday',
    'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
    'f_steps:fitbit_steps_intraday_rapids_avgdurationactivebout:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday',

]

# Define bins and labels for classification
bins = [0, 70, 84, 90, 100]  # Bins for sleep efficiency
labels = [0, 1, 2, 3]  # Bin labels for classification

# Apply binning to the target column
sleep_data['efficiency_bin'] = pd.cut(
    sleep_data['f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday'],
    bins=bins,
    labels=labels,
    right=False,
).astype("category")

# Add previous bin as a feature for temporal dependency
sleep_data['previous_bin'] = sleep_data.groupby('pid')['efficiency_bin'].shift(1)

# Drop rows where `efficiency_bin` or `previous_bin` is NaN
sleep_data = sleep_data.dropna(subset=['efficiency_bin', 'previous_bin'])

# Convert bins to integers for calculation
sleep_data['efficiency_bin'] = sleep_data['efficiency_bin'].astype(int)
sleep_data['previous_bin'] = sleep_data['previous_bin'].astype(int)

# Initialize lists to store results for each LOSO iteration
accuracy_list = []

# Perform LOSO (Leave-One-Subject-Out) cross-validation
for pid in sleep_data['pid'].unique():
    # Split data into training (all participants except the current one) and testing (current participant)
    train_data = sleep_data[sleep_data['pid'] != pid]
    test_data = sleep_data[sleep_data['pid'] == pid].sort_values(by='date')

    # Filter out rows with missing values for the selected features
    train_data = train_data.dropna(subset=selected_features)
    test_data = test_data.dropna(subset=selected_features)

    if len(test_data) < 5 or len(train_data) < 5:
        # Skip participants with too little data
        print(f"Skipping PID {pid} due to insufficient data")
        continue

    # Define X (features) and y (bin change) for train and test sets
    train_data['bin_change'] = (
        train_data['efficiency_bin'] - train_data['previous_bin']
    )
    test_data['bin_change'] = (
        test_data['efficiency_bin'] - test_data['previous_bin']
    )

    X_train = train_data[selected_features + ['previous_bin']]
    y_train = train_data['bin_change']
    X_test = test_data[selected_features + ['previous_bin']]
    y_test = test_data['bin_change']

    # Train the Random Forest Classifier model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy for the current left-out participant
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_list.append(accuracy)

    # # Output result for the current participant
    # print(f"PID: {pid}, Accuracy: {accuracy}")
    # print(classification_report(y_test, y_pred, zero_division=0))

# Calculate the overall average accuracy across all participants
average_accuracy_all = np.mean(accuracy_list)

# Print the overall results
print(f"\nAverage Accuracy across all LOSO iterations for bin change prediction: {average_accuracy_all}")



Average Accuracy across all LOSO iterations for bin change prediction: 0.9745151488125117


In [ ]:
df_anomaly= pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')

NameError: name 'pd' is not defined

In [ ]:
# Find column names that contain the word 'sleep'
sleep_columns = [col for col in df_anomaly.columns if 'sleep' in col.lower()]

# Print the result
print("Columns containing the word 'sleep':")
print(sleep_columns)


Columns containing the word 'sleep':
['f_slp:fitbit_sleep_intraday_rapids_avgdurationasleepunifiedmain:allday', 'f_slp:fitbit_sleep_intraday_rapids_avgdurationasleepunifiedmain_norm:allday', 'f_slp:fitbit_sleep_intraday_rapids_avgdurationawakeunifiedmain:allday', 'f_slp:fitbit_sleep_intraday_rapids_avgdurationawakeunifiedmain_norm:allday', 'f_slp:fitbit_sleep_intraday_rapids_countepisodeasleepunifiedmain:allday', 'f_slp:fitbit_sleep_intraday_rapids_countepisodeasleepunifiedmain_norm:allday', 'f_slp:fitbit_sleep_intraday_rapids_countepisodeawakeunifiedmain:allday', 'f_slp:fitbit_sleep_intraday_rapids_countepisodeawakeunifiedmain_norm:allday', 'f_slp:fitbit_sleep_intraday_rapids_maxdurationasleepunifiedmain:allday', 'f_slp:fitbit_sleep_intraday_rapids_maxdurationasleepunifiedmain_norm:allday', 'f_slp:fitbit_sleep_intraday_rapids_maxdurationawakeunifiedmain:allday', 'f_slp:fitbit_sleep_intraday_rapids_maxdurationawakeunifiedmain_norm:allday', 'f_slp:fitbit_sleep_intraday_rapids_mediandura

In [ ]:
columns_to_analyze = [
    'f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain:allday',
    'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
    'f_steps:fitbit_steps_intraday_rapids_avgdurationactivebout:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday',
    'anomaly'
]

# Filter relevant columns
df_filtered = df_anomaly[columns_to_analyze]

# Compute correlations with the 'anomaly' column
correlations = df_filtered.corr()['anomaly'].drop('anomaly')

# Display correlations
print("Correlations with anomaly:")
print(correlations)

Correlations with anomaly:
f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain:allday        0.400016
f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday               0.038017
f_steps:fitbit_steps_intraday_rapids_avgdurationactivebout:allday    0.019842
f_screen:phone_screen_rapids_avgdurationunlock:allday                0.003072
f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday    0.006507
Name: anomaly, dtype: float64


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import classification_report

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')

# Filter the first 20 participants
df = df[df['pid'].isin(df['pid'].unique()[:20])]

# Define features
selected_features = [
    'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
    'f_steps:fitbit_steps_intraday_rapids_avgdurationactivebout:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday',
]

selected_features_N1 = [
    'f_slp:fitbit_sleep_intraday_rapids_avgdurationasleepunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_avgdurationasleepunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_avgdurationawakeunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_avgdurationawakeunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_countepisodeasleepunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_countepisodeasleepunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_countepisodeawakeunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_countepisodeawakeunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_maxdurationasleepunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_maxdurationasleepunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_maxdurationawakeunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_maxdurationawakeunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_mediandurationasleepunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_mediandurationasleepunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_mediandurationawakeunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_mediandurationawakeunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_mindurationasleepunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_mindurationasleepunifiedmain_norm:allday',
    'f_slp:fitbit_sleep_intraday_rapids_mindurationawakeunifiedmain:allday',
    'f_slp:fitbit_sleep_intraday_rapids_mindurationawakeunifiedmain_norm:allday',
]

# Add lagged features for N1 (previous day)
for feature in selected_features_N1:
    df[f"{feature}_N1"] = df.groupby('pid')[feature].shift(1)



# Combine current and lagged features
all_features = selected_features + [f"{feature}_N1" for feature in selected_features_N1]
threshold = 0.5
df = df.loc[:, df.isnull().mean() <= threshold]
# Target variable
X = df[all_features]
y = df['anomaly']
groups = df['pid']  # Group identifier for LOSO

# Initialize LOSO cross-validator
logo = LeaveOneGroupOut()

# Initialize model and results
clf = RandomForestClassifier(random_state=42)
y_true, y_pred = [], []

# LOSO Cross-Validation
for train_index, test_index in logo.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Test the model
    preds = clf.predict(X_test)

    # Collect results
    y_true.extend(y_test)
    y_pred.extend(preds)

# Evaluate the results
print("Classification Report (LOSO):")
print(classification_report(y_true, y_pred))


Classification Report (LOSO):
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1395
           1       0.00      0.00      0.00        60

    accuracy                           0.96      1455
   macro avg       0.48      0.50      0.49      1455
weighted avg       0.92      0.96      0.94      1455



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Autoencoder Anomaly Detector

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')

# Filter the first 20 participants
df = df[df['pid'].isin(df['pid'].unique()[:20])]

# Define features
selected_features = [
    'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
    'f_steps:fitbit_steps_intraday_rapids_avgdurationactivebout:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday',
]

# Extract normal values (where anomaly == 0)
normal_data = df[df['anomaly'] == 0]

# Normalize features
scaler = StandardScaler()
X_normal = scaler.fit_transform(normal_data[selected_features])
X = scaler.transform(df[selected_features])
y = df['anomaly']
groups = df['pid']

# Build Autoencoder model
input_dim = X_normal.shape[1]
autoencoder = Sequential([
    Dense(16, activation='relu', input_dim=input_dim),
    Dense(8, activation='relu'),
    Dense(16, activation='relu'),
    Dense(input_dim, activation='linear')
])
autoencoder.compile(optimizer='adam', loss='mse')

# Train Autoencoder on normal data
autoencoder.fit(X_normal, X_normal, epochs=50, batch_size=32, shuffle=True, verbose=0)

# Leave-One-Subject-Out Cross-Validation
logo = LeaveOneGroupOut()
y_true, y_pred = [], []

# Define threshold for anomaly detection based on reconstruction error
reconstruction_errors = autoencoder.predict(X_normal) - X_normal
threshold = np.mean(np.square(reconstruction_errors)) + 2 * np.std(np.square(reconstruction_errors))

for train_index, test_index in logo.split(X, y, groups):
    X_test, y_test = X[test_index], y.iloc[test_index]

    # Calculate reconstruction errors
    reconstructions = autoencoder.predict(X_test)
    errors = np.mean(np.square(X_test - reconstructions), axis=1)

    # Predict anomalies based on reconstruction error
    y_test_pred = (errors > threshold).astype(int)

    # Collect results
    y_true.extend(y_test)
    y_pred.extend(y_test_pred)

# Evaluate the results
print("Classification Report (LOSO):")
print(classification_report(y_true, y_pred))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Classification Report (LOSO):
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1395
           1       0.04      0.02      0.02        60

    acc

# ISO Forest Anomaly Detector

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')

# Filter the first 20 participants
df = df[df['pid'].isin(df['pid'].unique()[:100])]

# Define features
selected_features = [
'f_steps:fitbit_steps_summary_rapids_minsumsteps:allday'  ,
'f_steps:fitbit_steps_summary_rapids_mediansumsteps:allday'    ,
'f_steps:fitbit_steps_summary_rapids_maxsumsteps:allday' ,
'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
'f_steps:fitbit_steps_intraday_rapids_sumsteps:allday',
'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_home:allday'  ,
'f_screen:phone_screen_rapids_firstuseafter00unlock:allday',
'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_living:allday',
'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_greens:allday'    ,
'f_slp:fitbit_sleep_summary_rapids_firstwaketimemain:allday' ,
'f_slp:fitbit_sleep_summary_rapids_lastwaketimemain:allday',
'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_study:allday'  ,
'f_screen:phone_screen_rapids_sumdurationunlock:allday',
                      #new features


]

selected_features_N1 = [

    'f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday',
    'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
    'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
    'f_steps:fitbit_steps_summary_rapids_minsumsteps:allday'  ,
'f_steps:fitbit_steps_summary_rapids_mediansumsteps:allday'    ,
'f_steps:fitbit_steps_summary_rapids_maxsumsteps:allday' ,
'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
'f_steps:fitbit_steps_intraday_rapids_sumsteps:allday',
'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_home:allday'  ,
'f_screen:phone_screen_rapids_firstuseafter00unlock:allday',
'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_living:allday',
'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_greens:allday'    ,
'f_slp:fitbit_sleep_summary_rapids_firstwaketimemain:allday' ,
'f_slp:fitbit_sleep_summary_rapids_lastwaketimemain:allday',
'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_study:allday'  ,
'f_screen:phone_screen_rapids_sumdurationunlock:allday',

]


# Add a rolling mean of the previous 7 days for each feature
for feature in selected_features_N1:
    df[f"{feature}_7day_avg"] = (
        df.groupby('pid')[feature]
        .rolling(window=7, min_periods=1)  # Rolling window of 7 days, minimum 1 day for calculation
        .mean()
        .reset_index(0, drop=True)  # Reset index after rolling
    )


all_features = selected_features + [f"{feature}_7day_avg" for feature in selected_features_N1]

# Drop columns with more than 50% NaN
df = df.loc[:, df.isnull().mean() <= 0.5]

# Drop rows with NaN
df = df.dropna()

# Extract normal data for training Isolation Forest
normal_data = df[df['anomaly'] == 0]

# Normalize features
scaler = StandardScaler()
X_normal = scaler.fit_transform(normal_data[all_features])
X = scaler.transform(df[all_features])
y = df['anomaly']
groups = df['pid']

# Train Isolation Forest on normal data
iso_forest = IsolationForest(
    contamination=0.2,
    max_samples='auto',
    max_features=0.8,  # Use 80% of features in each tree
    n_estimators=200,
    random_state=42
)
iso_forest.fit(X_normal)


# Leave-One-Subject-Out Cross-Validation
logo = LeaveOneGroupOut()
y_true, y_pred = [], []

for train_index, test_index in logo.split(X, y, groups):
    X_test, y_test = X[test_index], y.iloc[test_index]

    # Predict anomalies (-1 is anomaly, 1 is normal)
    predictions = iso_forest.predict(X_test)

    # Convert predictions to binary: 1 for anomaly, 0 for normal
    y_test_pred = (predictions == -1).astype(int)

    # Collect results
    y_true.extend(y_test)
    y_pred.extend(y_test_pred)

# Evaluate the results
print("Classification Report (LOSO):")
print(classification_report(y_true, y_pred))
balanced_acc = balanced_accuracy_score(y_true, y_pred)

print(f"Balanced Accuracy Score (LOSO): {balanced_acc:.4f}")


Classification Report (LOSO):
              precision    recall  f1-score   support

           0       0.96      0.80      0.87      6767
           1       0.09      0.40      0.14       328

    accuracy                           0.78      7095
   macro avg       0.53      0.60      0.51      7095
weighted avg       0.92      0.78      0.84      7095

Balanced Accuracy Score (LOSO): 0.5997


# ISO with PCA

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.decomposition import PCA

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')

# Filter the first 100 participants
df = df[df['pid'].isin(df['pid'].unique()[:100])]

# Define features
selected_features = [
    'f_steps:fitbit_steps_summary_rapids_minsumsteps:allday',
    'f_steps:fitbit_steps_summary_rapids_mediansumsteps:allday',
    'f_steps:fitbit_steps_summary_rapids_maxsumsteps:allday',
    'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
    'f_steps:fitbit_steps_intraday_rapids_sumsteps:allday',
    'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_home:allday',
    'f_screen:phone_screen_rapids_firstuseafter00unlock:allday',
    'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_living:allday',
    'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_greens:allday',
    'f_slp:fitbit_sleep_summary_rapids_firstwaketimemain:allday',
    'f_slp:fitbit_sleep_summary_rapids_lastwaketimemain:allday',
    'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
    'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
    'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_study:allday',
    'f_screen:phone_screen_rapids_sumdurationunlock:allday',
]

selected_features_N1 = [
    'f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday',
    'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
    'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
]

# Add a rolling mean of the previous 7 days for each feature
for feature in selected_features_N1:
    df[f"{feature}_7day_avg"] = (
        df.groupby('pid')[feature]
        .rolling(window=7, min_periods=1)  # Rolling window of 7 days, minimum 1 day for calculation
        .mean()
        .reset_index(0, drop=True)  # Reset index after rolling
    )

# Combine current and lagged features
all_features = selected_features + [f"{feature}_7day_avg" for feature in selected_features_N1]

# Drop columns with more than 50% NaN
df = df.loc[:, df.isnull().mean() <= 0.5]

# Drop rows with NaN
df = df.dropna()

# Extract normal data for training Isolation Forest
normal_data = df[df['anomaly'] == 0]

# Normalize features
scaler = StandardScaler()
X_normal = scaler.fit_transform(normal_data[all_features])
X = scaler.transform(df[all_features])
y = df['anomaly']
groups = df['pid']

# Apply PCA to reduce dimensions
pca = PCA(n_components=10)  # Adjust the number of components as needed
X_pca = pca.fit_transform(X)

# Train Isolation Forest on PCA-transformed normal data
iso_forest = IsolationForest(
    contamination=0.2,
    max_samples='auto',
    max_features=0.8,  # Use 80% of features in each tree
    n_estimators=200,
    random_state=42
)
iso_forest.fit(X_pca)

# Leave-One-Subject-Out Cross-Validation
logo = LeaveOneGroupOut()
y_true, y_pred = [], []

for train_index, test_index in logo.split(X_pca, y, groups):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_test = y.iloc[test_index]

    # Train Isolation Forest on PCA-transformed training data
    iso_forest.fit(X_train)

    # Predict anomalies (-1 is anomaly, 1 is normal)
    predictions = iso_forest.predict(X_test)

    # Convert predictions to binary: 1 for anomaly, 0 for normal
    y_test_pred = (predictions == -1).astype(int)

    # Collect results
    y_true.extend(y_test)
    y_pred.extend(y_test_pred)

# Evaluate the results
print("Classification Report (LOSO with PCA):")
print(classification_report(y_true, y_pred))
balanced_acc = balanced_accuracy_score(y_true, y_pred)
print(f"Balanced Accuracy Score (LOSO with PCA): {balanced_acc:.4f}")


<ipython-input-128-74cd724d3d75>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{feature}_7day_avg"] = (
<ipython-input-128-74cd724d3d75>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{feature}_7day_avg"] = (
<ipython-input-128-74cd724d3d75>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

Classification Report (LOSO with PCA):
              precision    recall  f1-score   support

           0       0.96      0.80      0.87      6767
           1       0.07      0.31      0.11       328

    accuracy                           0.78      7095
   macro avg       0.51      0.56      0.49      7095
weighted avg       0.92      0.78      0.84      7095

Balanced Accuracy Score (LOSO with PCA): 0.5551


In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import balanced_accuracy_score

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')

# Filter the first 20 participants
df = df[df['pid'].isin(df['pid'].unique()[:50])]

# Define features
selected_features = [
    'f_steps:fitbit_steps_summary_rapids_minsumsteps:allday',
    # 'f_steps:fitbit_steps_summary_rapids_mediansumsteps:allday',
    # 'f_steps:fitbit_steps_summary_rapids_maxsumsteps:allday',
    # 'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
    # 'f_steps:fitbit_steps_intraday_rapids_sumsteps:allday',
    # 'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_home:allday',
    # 'f_screen:phone_screen_rapids_firstuseafter00unlock:allday',
    # 'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_living:allday',
    'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_greens:allday',
    # 'f_slp:fitbit_sleep_summary_rapids_firstwaketimemain:allday',
    # 'f_slp:fitbit_sleep_summary_rapids_lastwaketimemain:allday',
    # 'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
    # 'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
    # 'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_study:allday',
    # 'f_screen:phone_screen_rapids_sumdurationunlock:allday',
    # 'f_slp:fitbit_sleep_summary_rapids_firstwaketimemain:allday',
    # 'f_slp:fitbit_sleep_summary_rapids_lastwaketimemain:allday',
]

selected_features_N1 = [
    'f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday',
    'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
    'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
]

# Add a rolling mean of the previous 7 days for each feature
for feature in selected_features_N1:
    df[f"{feature}_7day_avg"] = (
        df.groupby('pid')[feature]
        .shift(1)
        .rolling(window=1, min_periods=1)  # Rolling window of 7 days, minimum 1 day for calculation
        .mean()
        .reset_index(0, drop=True)  # Reset index after rolling
    )

# Apply log transformation to handle high variance features
log_transformed_features = []
for feature in selected_features + [f"{feature}_7day_avg" for feature in selected_features_N1]:
    df[f"{feature}_log"] = np.log1p(df[feature])  # log1p to handle zero values
    log_transformed_features.append(f"{feature}_log")

# Drop columns with more than 50% NaN
df = df.loc[:, df.isnull().mean() <= 0.5]

# Drop rows with NaN
df = df.dropna()

# Extract normal data for training Isolation Forest
normal_data = df[df['anomaly'] == 0]

# Normalize features
scaler = StandardScaler()
X_normal = scaler.fit_transform(normal_data[log_transformed_features])
X = scaler.transform(df[log_transformed_features])
y = df['anomaly']
groups = df['pid']

# Train Isolation Forest on normal data
iso_forest = IsolationForest(contamination=0.05, random_state=42)

# Leave-One-Subject-Out Cross-Validation
logo = LeaveOneGroupOut()
y_true, y_pred = [], []

for train_index, test_index in logo.split(X, y, groups):
    X_train, y_train = X[train_index], y.iloc[train_index]
    X_test, y_test = X[test_index], y.iloc[test_index]

    iso_forest.fit(X_train)

    # Predict anomalies (-1 is anomaly, 1 is normal)
    predictions = iso_forest.predict(X_test)

    # Convert predictions to binary: 1 for anomaly, 0 for normal
    y_test_pred = (predictions == -1).astype(int)

    # Collect results
    y_true.extend(y_test)
    y_pred.extend(y_test_pred)

# Evaluate the results
print("Classification Report (LOSO):")
print(classification_report(y_true, y_pred))
balanced_acc = balanced_accuracy_score(y_true, y_pred)

print(f"Balanced Accuracy Score (LOSO): {balanced_acc:.4f}")


Classification Report (LOSO):
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      3435
           1       0.08      0.11      0.09       148

    accuracy                           0.91      3583
   macro avg       0.52      0.53      0.52      3583
weighted avg       0.92      0.91      0.92      3583

Balanced Accuracy Score (LOSO): 0.5279


In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')

# Filter first 20 participants
df = df[df['pid'].isin(df['pid'].unique()[:20])]

# Define features
selected_features = [
    'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
    'f_steps:fitbit_steps_intraday_rapids_avgdurationactivebout:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock:allday',
    'f_screen:phone_screen_rapids_avgdurationunlock_locmap_home:allday',
]

# Add statistical features
for feature in selected_features:
    df[f"{feature}_mean"] = df.groupby('pid')[feature].rolling(3).mean().reset_index(0, drop=True)
    df[f"{feature}_std"] = df.groupby('pid')[feature].rolling(3).std().reset_index(0, drop=True)

# Update feature list
all_features = selected_features + [f"{feature}_mean" for feature in selected_features]

# Drop columns with >50% NaN
normal_data = df[df['anomaly'] == 0]
df = df.loc[:, df.isnull().mean() <= 0.5]
df = df.dropna()
scaler = StandardScaler()
X_normal = scaler.fit_transform(normal_data[all_features])
X = scaler.transform(df[all_features])
# Extract normal data


# Initialize LOSO
logo = LeaveOneGroupOut()
y_true, y_pred = [], []

for train_index, test_index in logo.split(df[all_features], df['anomaly'], df['pid']):
    X_train, X_test = df[all_features].iloc[train_index], df[all_features].iloc[test_index]
    y_test = df['anomaly'].iloc[test_index]

    # Normalize
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train Isolation Forest
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    iso_forest.fit(X_train)

    # Predict anomalies
    anomaly_scores = -iso_forest.decision_function(X_test)
    threshold = np.percentile(anomaly_scores, 90)
    y_test_pred = (anomaly_scores > threshold).astype(int)

    y_true.extend(y_test)
    y_pred.extend(y_test_pred)

# Evaluate
print("Classification Report (LOSO):")
print(classification_report(y_true, y_pred))


Classification Report (LOSO):
              precision    recall  f1-score   support

           0       0.96      0.89      0.92      1355
           1       0.02      0.05      0.03        60

    accuracy                           0.86      1415
   macro avg       0.49      0.47      0.48      1415
weighted avg       0.92      0.86      0.89      1415



In [ ]:

# Filter rows where 'bin_change' column is less than 0
df_filtered = df[df['bin_change'] < 0]

# Select only numeric columns for variance calculation
numeric_data = df_filtered.select_dtypes(include=[float, int])

# Calculate variance for numeric columns
feature_deviation = numeric_data.var().drop('bin_change', errors='ignore')  # Exclude 'bin_change' if present
top_5_deviations = feature_deviation.sort_values(ascending=False).head(30)

# Display the top 5 features with the highest deviation
print("Top 5 features with the highest deviation where bin_change < 0:")
print(top_5_deviations)

Top 5 features with the highest deviation where bin_change < 0:
f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday                       2.662350e+07
f_steps:fitbit_steps_intraday_rapids_sumsteps:allday                         2.662350e+07
f_steps:fitbit_steps_summary_rapids_maxsumsteps:allday                       2.662350e+07
f_steps:fitbit_steps_summary_rapids_mediansumsteps:allday                    2.662350e+07
f_steps:fitbit_steps_summary_rapids_minsumsteps:allday                       2.662350e+07
f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_living:allday      1.413362e+05
f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_home:allday        1.117041e+05
f_screen:phone_screen_rapids_firstuseafter00unlock:allday                    6.230074e+04
f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_greens:allday      5.092067e+04
f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday                    2.934716e+04
f_screen:phone_screen_rapids_firstus

# ZSCORE Calculation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from collections import Counter

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')

# Filter the first 40 participants
df = df[df['pid'].isin(df['pid'].unique()[:40])]

# Total rows where anomaly = 1
total_anomaly_1 = df[df['anomaly'] == 1].shape[0]
print(f"Total rows with anomaly = 1: {total_anomaly_1}")

# Identify all numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()

# Exclude non-feature columns like 'anomaly' and 'pid' if present
numeric_columns = [col for col in numeric_columns if col not in ['anomaly', 'pid']]

# Group data by `pid`
grouped = df.groupby('pid')

# Initialize results
deviated_features_anomaly_1 = {feature: [] for feature in numeric_columns}

# Perform anomaly detection for each participant
for pid, group in grouped:
    # Filter rows where anomaly == 1
    anomaly_data = group[group['anomaly'] == 1]

    if not anomaly_data.empty:
        for feature in numeric_columns:
            # Calculate mean and std for the feature
            mean = group[feature].mean()
            std = group[feature].std()

            # Avoid division by zero
            std = std if std != 0 else 1e-8

            # Check deviation from mean for each value
            deviations = np.abs((anomaly_data[feature] - mean) / std) > 1  # Z-score threshold: 1
            if deviations.any():
                deviated_features_anomaly_1[feature].append(pid)

# Count the number of participants (pids) where each feature deviates for anomaly = 1
feature_count_anomaly_1 = Counter({feature: len(set(pids)) for feature, pids in deviated_features_anomaly_1.items()})

# Get the top 8 features with the most occurrences across participants for anomaly = 1
top_features_anomaly_1 = feature_count_anomaly_1.most_common(30)

# Print the top 8 features
print("Top 8 features with the most deviations for anomaly = 1 across participants:")
for feature, count in top_features_anomaly_1:
    print(f"{feature}")


Total rows with anomaly = 1: 139
Top 8 features with the most deviations for anomaly = 1 across participants:
f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday
f_slp:fitbit_sleep_summary_rapids_avgefficiencymain_norm:allday
efficiency_bin
bin_change
f_slp:fitbit_sleep_summary_rapids_avgdurationawakemain:allday
f_slp:fitbit_sleep_summary_rapids_avgdurationawakemain_norm:allday
f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain:allday
f_slp:fitbit_sleep_summary_rapids_sumdurationawakemain_norm:allday
f_slp:fitbit_sleep_summary_rapids_avgdurationinbedmain_norm:allday
f_screen:phone_screen_rapids_maxdurationunlock_locmap_home_norm:allday
f_slp:fitbit_sleep_summary_rapids_avgdurationinbedmain:allday
f_slp:fitbit_sleep_summary_rapids_firstbedtimemain_norm:allday
f_slp:fitbit_sleep_summary_rapids_sumdurationinbedmain:allday
f_slp:fitbit_sleep_summary_rapids_sumdurationinbedmain_norm:allday
f_steps:fitbit_steps_intraday_rapids_maxdurationsedentarybout:allday
f_steps:fitbit_steps_i

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneGroupOut

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')
df = df[df['pid'].isin(df['pid'].unique()[:30])]
# Define features
selected_features = [
    'f_steps:fitbit_steps_summary_rapids_minsumsteps:allday',
    'f_screen:phone_screen_rapids_maxdurationunlock_locmap_home_norm:allday',
    'f_screen:phone_screen_rapids_countepisodeunlock:allday',
  'f_screen:phone_screen_rapids_sumdurationunlock:allday',
    #'f_steps:fitbit_steps_summary_rapids_mediansumsteps:allday',
    #'f_steps:fitbit_steps_summary_rapids_maxsumsteps:allday',
    # 'f_steps:fitbit_steps_summary_rapids_avgsumsteps:allday',
    # 'f_steps:fitbit_steps_intraday_rapids_sumsteps:allday',
    # 'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_home:allday',
    # 'f_screen:phone_screen_rapids_firstuseafter00unlock:allday',
    # 'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_living:allday',
    'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_greens:allday',
]

selected_features_N1 = [
    'f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday',
    'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
    'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
]
for feature_N1 in selected_features_N1:
    df[f"{feature_N1}_7day_avg"] = (
        df.groupby('pid')[feature_N1]
        .shift(1)  # Shift to exclude the current row
        .rolling(window=1, min_periods=1)  # Rolling window of 7 (now excluding current row)
        .mean()
        .reset_index(0, drop=True)
    )

for feature in selected_features:
    df[f"{feature}_7day_avg"] = (
        df.groupby('pid')[feature]
        .shift(1)  # Shift to exclude the current row
        .rolling(window=1, min_periods=1)  # Rolling window of 7 (now excluding current row)
        .mean()
        .reset_index(0, drop=True)
    )

all_features = selected_features + [f"{feature_N1}_7day_avg" for feature_N1 in selected_features_N1]+ [f"{feature}_7day_avg" for feature in selected_features]
# Add features for the previous day (N-1)
# for feature in selected_features_N1:
#     df[f"N1_{feature}"] = df.groupby('pid')[feature].shift(1)

# all_features = selected_features + [f"N1_{feature}" for feature in selected_features_N1]

# Drop rows with NaN
df = df.dropna(subset=all_features + ['anomaly'])

# Prepare data
X = df[all_features].to_numpy()
y = df['anomaly'].to_numpy()
groups = df['pid'].to_numpy()

# Leave-One-Subject-Out Cross-Validation
logo = LeaveOneGroupOut()
y_true, y_pred_rf = [], []

for train_index, test_index in logo.split(X, y, groups):
    # Split data
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Normalize data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Oversample with SMOTE
    smote = SMOTE(sampling_strategy='minority', random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    # Train Random Forest
    rf_model = RandomForestClassifier(
    n_estimators=100, random_state=42, class_weight={0: 1, 1: 5}
)
    rf_model.fit(X_train, y_train)

    # Predict
    rf_predictions = rf_model.predict(X_test)

    # Collect results
    y_true.extend(y_test)
    y_pred_rf.extend(rf_predictions)

# Evaluate Random Forest Model
print("Classification Report (Random Forest):")
print(classification_report(y_true, y_pred_rf))


Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      2039
           1       0.06      0.04      0.05       119

    accuracy                           0.91      2158
   macro avg       0.50      0.50      0.50      2158
weighted avg       0.90      0.91      0.90      2158



In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/CSE 570 Wireless Project/final_dataset/anomaly_detect_for_all_participants.csv')
df = df[df['pid'].isin(df['pid'].unique()[:30])]

# Define features
selected_features = [
    'f_steps:fitbit_steps_summary_rapids_minsumsteps:allday',
    'f_screen:phone_screen_rapids_firstuseafter00unlock_locmap_greens:allday',
]

selected_features_N1 = [
    'f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:allday',
    'f_slp:fitbit_sleep_summary_rapids_firstbedtimemain:allday',
    'f_slp:fitbit_sleep_summary_rapids_lastbedtimemain:allday',
]

# Create 7-day rolling averages for N1 features
for feature_N1 in selected_features_N1:
    df[f"{feature_N1}_7day_avg"] = (
        df.groupby('pid')[feature_N1]
        .shift(1)  # Exclude the current row
        .rolling(window=1, min_periods=1)
        .mean()
        .reset_index(0, drop=True)
    )

for feature in selected_features:
    df[f"{feature}_7day_avg"] = (
        df.groupby('pid')[feature]
        .shift(1)  # Exclude the current row
        .rolling(window=1, min_periods=1)
        .mean()
        .reset_index(0, drop=True)
    )

all_features = selected_features + [f"{feature_N1}_7day_avg" for feature_N1 in selected_features_N1] + [f"{feature}_7day_avg" for feature in selected_features]

# Drop rows with NaN
df = df.dropna(subset=all_features + ['efficiency_bin'])

# Prepare data
X = df[all_features].to_numpy()
y = df['efficiency_bin'].to_numpy()

# Split into train and test (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Oversample with SMOTE
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Train Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100, random_state=42, class_weight={0: 10, 1: 5,2:3,3:1}
)
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Evaluate Random Forest Model
print("Classification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf))


Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00        13
           3       0.93      0.98      0.95       201

    accuracy                           0.91       216
   macro avg       0.23      0.25      0.24       216
weighted avg       0.86      0.91      0.89       216



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
